**Learning outcomes**

* Explain how global information and promises are useful for algorithm design.

In [1]:
import numpy as np
import pennylane as qml

The oracle once more produces an undetectable phase:
$$\Ket{s_1} + \Ket{s_2} \rightarrow -(\Ket{s_1} + \Ket{s_2})$$
If there are an *odd* number of solutions, then even if we miss a number of pairs solutions, there is guaranteed to be one left over.
The pair-testing algorithm also tells us that, conversely, global assumptions about the function (e.g., that the number of solutions is odd) can lead to very different performance guarantees for our algorithms.
![circuit](./images/A.3.1.png)



**Codercise A.4.1.** Implement the circuit above, but now for `how_many` solutions instead of one. You will first need to implement the multi-solution version of the oracle matrix,`multisol_oracle_matrix(combos)`, which takes a list of bit strings (representing different solutions) and returns the associated oracle in matrix form.

In [12]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)
def get_value(value,):
    return value
def multisol_oracle_matrix(combos):
    """Return the oracle matrix for a set of solutions.

    Args:
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    indices = [np.ravel_multi_index(combo, [2]*len(combo)) for combo in combos]
    my_array = np.identity(2**len(combos[0])) # Create the identity matrix
    for i in indices:
        my_array[i,i] = -1
    return my_array

@qml.qnode(dev)
def multisol_pair_circuit(x_tilde, combos):
    """Implements the circuit for testing a pair of combinations labelled by x_tilde.

    Args:
        x_tilde (list[int]): An (n_bits - 1)-bit string labelling the pair to test.
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    oracle = multisol_oracle_matrix(combos)

    qml.Hadamard(n_bits-1)
    qml.QubitUnitary(oracle, wires=[i for i in range(n_bits)])
    qml.Hadamard(n_bits-1)

    return qml.probs(wires=n_bits-1)
x_tilde = [1,1,1]
combos = [ ]
multisol_pair_circuit(x_tilde, combos)

**Codercise A.4.2.** Use `multisol_pair_circuit(x_tilde, combos)` to create a routine for checking the parity of a lock.

In [ ]:
def parity_checker(combos):
    """Use multisol_pair_circuit to determine the parity of a solution set.

    Args:
        combos (list[list[int]]): A list of secret combinations.

    Returns:
        int: The parity of the solution set.
    """
    parity = 0
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]

    for x_tilde in x_tildes:
        # IMPLEMENT PARITY COUNTING ALGORITHM
        p0, p1  = multisol_pair_circuit(x_tilde, combos)
        if np.isclose(1, p1):
            parity+=1

    return parity % 2
